In [36]:
import os
import re
from pyspark.sql import SparkSession, Window
from google.colab import drive
from pyspark.sql import functions as f
from pyspark.sql.types import DoubleType, ArrayType, StringType

Установка Spark

In [37]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

!wget -q https://downloads.apache.org/spark/spark-3.5.5/spark-3.5.5-bin-hadoop3.tgz
!tar xf spark-3.5.5-bin-hadoop3.tgz

!pip install -q findspark
import findspark

Задание переменных окружения

In [38]:
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.5.5-bin-hadoop3"

Создание и настройка сессии Spark

In [39]:
findspark.init()
spark = SparkSession.builder.master("local[*]").config("spark.jars.packages", "com.databricks:spark-xml_2.12:0.14.0").getOrCreate()
spark.conf.set("spark.sql.repl.eagerEval.enabled", True)
spark

Загрузка данных

In [40]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [41]:
# датафрейм с языками программирования
languages = spark.read.csv('/content/drive/MyDrive/Colab Notebooks/Большие данные/programming-languages.csv', header=True, inferSchema=True)
languages.printSchema()
languages.show(5, truncate=False)

root
 |-- name: string (nullable = true)
 |-- wikipedia_url: string (nullable = true)

+----------+---------------------------------------------------------+
|name      |wikipedia_url                                            |
+----------+---------------------------------------------------------+
|A# .NET   |https://en.wikipedia.org/wiki/A_Sharp_(.NET)             |
|A# (Axiom)|https://en.wikipedia.org/wiki/A_Sharp_(Axiom)            |
|A-0 System|https://en.wikipedia.org/wiki/A-0_System                 |
|A+        |https://en.wikipedia.org/wiki/A%2B_(programming_language)|
|A++       |https://en.wikipedia.org/wiki/A%2B%2B                    |
+----------+---------------------------------------------------------+
only showing top 5 rows



In [42]:
# датафрейм с данными о постах
posts = spark.read.format('xml').options(rowTag='row').load('/content/drive/MyDrive/Colab Notebooks/Большие данные/posts_sample.xml')
posts.printSchema()
posts.show(5, truncate=False)

root
 |-- _AcceptedAnswerId: long (nullable = true)
 |-- _AnswerCount: long (nullable = true)
 |-- _Body: string (nullable = true)
 |-- _ClosedDate: timestamp (nullable = true)
 |-- _CommentCount: long (nullable = true)
 |-- _CommunityOwnedDate: timestamp (nullable = true)
 |-- _CreationDate: timestamp (nullable = true)
 |-- _FavoriteCount: long (nullable = true)
 |-- _Id: long (nullable = true)
 |-- _LastActivityDate: timestamp (nullable = true)
 |-- _LastEditDate: timestamp (nullable = true)
 |-- _LastEditorDisplayName: string (nullable = true)
 |-- _LastEditorUserId: long (nullable = true)
 |-- _OwnerDisplayName: string (nullable = true)
 |-- _OwnerUserId: long (nullable = true)
 |-- _ParentId: long (nullable = true)
 |-- _PostTypeId: long (nullable = true)
 |-- _Score: long (nullable = true)
 |-- _Tags: string (nullable = true)
 |-- _Title: string (nullable = true)
 |-- _ViewCount: long (nullable = true)

+-----------------+------------+---------------------------------------------

#**Сформировать отчёт с информацией о 10 наиболее популярных языках программирования по итогам года за период с 2010 по 2020 годы. Отчёт будет отражать динамику изменения популярности языков программирования и представлять собой набор таблиц "топ-10" для каждого года. Получившийся отчёт сохранить в формате Apache Parquet.**

In [43]:
# оставляем только необходимые столбцы и года
cropped_posts = posts.selectExpr("year(_LastActivityDate) as year", "_Tags as tag", "_ViewCount as views").na.drop()
cropped_posts = cropped_posts.filter((cropped_posts.year >= 2010) & (cropped_posts.year <= 2020))

# оставляем только названия языков программирования
cropped_languages = languages.select("name")

# функция для преобразования строки тегов в список тегов
def tags_to_list(tags):
    return re.findall(r'<(.+?)>', tags)

tags_to_list_udf = f.udf(tags_to_list, ArrayType(StringType()))

# применяем пользовательскую функцию к столбцу тегов
cropped_posts = cropped_posts.withColumn("tag", tags_to_list_udf(cropped_posts["tag"]))

# разбиваем список тегов построчно
tags = cropped_posts.withColumn("tag", f.explode(cropped_posts["tag"]))

tags.show(10, truncate=False)
cropped_languages.show(10, truncate=False)

+----+-------------------+------+
|year|tag                |views |
+----+-------------------+------+
|2019|c#                 |42817 |
|2019|floating-point     |42817 |
|2019|type-conversion    |42817 |
|2019|double             |42817 |
|2019|decimal            |42817 |
|2019|html               |18214 |
|2019|css                |18214 |
|2019|internet-explorer-7|18214 |
|2019|c#                 |555183|
|2019|.net               |555183|
+----+-------------------+------+
only showing top 10 rows

+----------+
|name      |
+----------+
|A# .NET   |
|A# (Axiom)|
|A-0 System|
|A+        |
|A++       |
|ABAP      |
|ABC       |
|ABC ALGOL |
|ABSET     |
|ABSYS     |
+----------+
only showing top 10 rows



In [44]:
# приводим теги и языки к нижнему регистру и удаляем пробелы для дальнейшего сопоставления
normalized_tags = tags.withColumn("tag", f.regexp_replace(f.lower(f.col("tag")), "\\s+", ""))
normalized_languages = cropped_languages.withColumn("name", f.regexp_replace(f.lower(f.col("name")), "\\s+", ""))

# оставляем только те теги, что входят в список языков
languages_info = normalized_tags.join(normalized_languages, normalized_tags.tag == f.expr(f"substring(name, 1, length(tag))"), "inner").dropDuplicates(["year", "tag", "views"]).drop("name")
languages_info = languages_info.withColumnRenamed("tag", "language")

languages_info.show(10, truncate=False)

+----+------------+-----+
|year|language    |views|
+----+------------+-----+
|2016|javascript  |671  |
|2015|c++         |246  |
|2011|actionscript|3342 |
|2011|c           |252  |
|2011|javascript  |207  |
|2014|ruby        |29278|
|2012|javascript  |607  |
|2018|qt          |5658 |
|2015|powershell  |31   |
|2016|powershell  |192  |
+----+------------+-----+
only showing top 10 rows



In [45]:
# для каждого языка считаем общее количество просмотров по годам
popular_languages = languages_info.groupBy("year", "language").agg(f.sum("views").alias("all_views"))

# нумеруем языки по годам, сортируя по количеству просмотров
popular_languages = popular_languages.withColumn("number", f.row_number().over(Window.partitionBy("year").orderBy(f.col("all_views").desc())))

# оставляем топ-10
top_languages = popular_languages.filter(f.col("number") <= 10)

top_languages.show(top_languages.count(),truncate=False)

# сохраняем отчёт в формате Apache Parquet, разбивая по годам
top_languages.write.mode("overwrite").partitionBy("year").parquet("/content/drive/MyDrive/Colab Notebooks/Большие данные/top_languages.parquet")

+----+-----------+---------+------+
|year|language   |all_views|number|
+----+-----------+---------+------+
|2010|c#         |128597   |1     |
|2010|java       |53333    |2     |
|2010|matlab     |51865    |3     |
|2010|objective-c|43878    |4     |
|2010|php        |39730    |5     |
|2010|javascript |37059    |6     |
|2010|python     |25930    |7     |
|2010|c++        |22568    |8     |
|2010|ruby       |15864    |9     |
|2010|c          |13810    |10    |
|2011|c#         |237943   |1     |
|2011|java       |121118   |2     |
|2011|python     |89637    |3     |
|2011|c          |73116    |4     |
|2011|php        |67260    |5     |
|2011|c++        |61741    |6     |
|2011|javascript |61631    |7     |
|2011|objective-c|54686    |8     |
|2011|object     |33084    |9     |
|2011|qt         |15835    |10    |
|2012|php        |303420   |1     |
|2012|c#         |291478   |2     |
|2012|java       |272122   |3     |
|2012|python     |220014   |4     |
|2012|javascript |204366   |

In [46]:
# проверяем, было ли успешным сохранение
parquet_top = spark.read.parquet("/content/drive/MyDrive/Colab Notebooks/Большие данные/top_languages.parquet")
parquet_top.show(parquet_top.count(), truncate=False)

+-----------+---------+------+----+
|language   |all_views|number|year|
+-----------+---------+------+----+
|python     |3405147  |1     |2019|
|php        |1512707  |2     |2019|
|c#         |1316086  |3     |2019|
|javascript |875256   |4     |2019|
|java       |568606   |5     |2019|
|objective-c|430671   |6     |2019|
|windows    |243407   |7     |2019|
|c++        |135293   |8     |2019|
|swift      |100651   |9     |2019|
|r          |51738    |10    |2019|
|php        |303420   |1     |2012|
|c#         |291478   |2     |2012|
|java       |272122   |3     |2012|
|python     |220014   |4     |2012|
|javascript |204366   |5     |2012|
|c++        |71192    |6     |2012|
|pip        |58370    |7     |2012|
|objective-c|50825    |8     |2012|
|bash       |36183    |9     |2012|
|ruby       |32525    |10    |2012|
|c#         |406339   |1     |2016|
|javascript |380128   |2     |2016|
|java       |341029   |3     |2016|
|python     |217403   |4     |2016|
|php        |148587   |5    